In [1]:
import numpy as np
import torch

from transformers import GPT2Tokenizer, GPT2LMHeadModel

## Importance sampling

In [2]:
from utils import set_seed, create_history, create_model_kwargs
from tqdm import tqdm
set_seed(42)

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
def get_imp_sample(model, tokenizer, input_str, avoid_terms, n, seed, max_num_tokens=5, debug=False):
    from sampling import importance_sampling, naive_sampling
    from utils import set_seed, create_history, create_model_kwargs
    
    # Parse input and set seeds for reproducibility
    set_seed(seed)
    bos_token_id = tokenizer.bos_token_id or model.config.decoder_start_token_id
    input_ids = tokenizer(input_str, return_tensors="pt", add_special_tokens=False).input_ids

    # some models have different representations for the same term depending on
    # whether they are preceeded with a space or not (we account for that by
    # producing a double-length list of terms to avoid, so that we can)
    # this list is actually reduced to the unique terms within the sampling methods.
    avoid_terms_ids = tokenizer(avoid_terms, add_special_tokens=False).input_ids

    # History (or past observations) and model_kwargs will be the same for all queries
    history = create_history(n, input_ids, bos_token_id)

    probabilities, samples, _ = importance_sampling(
        avoid_term_ids=avoid_terms_ids,
        **create_model_kwargs(history, model, tokenizer),
        max_num_tokens=max_num_tokens,
        model=model,
        tokenizer=tokenizer,
    )
    
    return probabilities, samples

In [5]:
SEEDS = np.unique(np.random.randint(0, 10**6, 10))
SEED = SEEDS[0]
MAX_NUM_TOKENS=5
print("using", len(SEEDS), "seeds")

# User definitions
model_name = "gpt2"
num_samples = 100

# ==========================================================
# Load models
tokenizer = GPT2Tokenizer.from_pretrained(model_name, model_max_length=512)
model = GPT2LMHeadModel.from_pretrained(model_name, pad_token_id=tokenizer.eos_token_id)

print("Total #sequences:", 10*num_samples)

using 10 seeds
Total #sequences: 1000


## Contextual Calibration

In [7]:
imp_sampling_kwargs = {
    "model": model,
    "tokenizer": tokenizer, 
    "n": num_samples,
    "seed": SEED,
    "max_num_tokens": 30,
}

### Marginalize content-free inputs

In [8]:
input_str = "Input: Subpar acting. Sentiment: Negative\nInput: Beautiful film. Sentiment: Positive\n Input:"
labels = [" Positive", " Negative"]

In [9]:
pos_prob, pos_samples = get_imp_sample(input_str=input_str, avoid_terms=labels[0], **imp_sampling_kwargs)
neg_prob, neg_samples = get_imp_sample(input_str=input_str, avoid_terms=labels[1], **imp_sampling_kwargs)

In [10]:
round(100-pos_prob.mean().item() * 100, 2)

20.24

In [11]:
tokenizer.batch_decode(pos_samples)

['Input: Subpar acting. Sentiment: Negative\nInput: Beautiful film. Sentiment: Positive\n Input: Excellent\nInput: Probably ruined by the terrible track start. Waiting for green to cancel for a "real" brajal. Rodal is sn',
 "Input: Subpar acting. Sentiment: Negative\nInput: Beautiful film. Sentiment: Positive\n Input: Graphic, but they wouldn't be USED for many reasons - materialism, troublesome subject matter, terrible writing, etc. However, this computer put",
 'Input: Subpar acting. Sentiment: Negative\nInput: Beautiful film. Sentiment: Positive\n Input: Impressive. Reportedly respectful of people. Good for stripping scene. Censors love the positive flick mindgames. Source/Transcription: Read',
 'Input: Subpar acting. Sentiment: Negative\nInput: Beautiful film. Sentiment: Positive\n Input: Love is Beautiful. Sentiment: Negative\nInput: You are very asking for it.\nSentiment: Negative\nOutput: Ultimate. Only for',
 'Input: Subpar acting. Sentiment: Negative\nInput: Beautiful film. Se

In [12]:
round(100-neg_prob.mean().item() * 100, 2)

29.64

In [13]:
test_input_str = f"{input_str} N/A Sentiment:"
test_input_ids = tokenizer(test_input_str, return_tensors="pt").input_ids

# Get probabilities of GPT2 model